# Instanciar Spark

In [1]:
import pyspark.sql.functions as f
from pyspark.sql.functions import col, from_json , explode, expr
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
import pandas as pd
import os

import requests, json

PATH = "C:\\PROJETOS\\ibge_integration\\RAW_DATA\\" # SUBSTITUA UM PATH PARA UM DE SUA PREFERENCIA OU UM BUCKET
PATH = "E:\\AMBIENTE DEV\\PROJETOS\\ibge_integration\\RAW_DATA\\" # SUBSTITUA UM PATH PARA UM DE SUA PREFERENCIA OU UM BUCKET


spark_session = SparkSession.builder \
    .appName("Ibge") \
    .config("spark.master", "local[*]") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
    .config("spark.executor.instances", "3") \
    .getOrCreate()

# SQL CONNECTION

In [2]:
# Configurações de conexão com o PostgreSQL
# Este item é totalmente editavel, para uma conexão de sua preferencia (Vamos fazer conforme o docker-compose que subimos)
url_database = "jdbc:postgresql://localhost:8085/bronze_data"
properties = {"user": "ibge", "password": "ibge", "driver": "org.postgresql.Driver"}

# engine = create_engine(url_database)

# BRONZE DATA - (Região, Estado e Municipio)

In [139]:
# LER O PARQUE DE REGIÕES E GRAVAR NO BANCO

df_regions = spark_session.read.parquet(f"{PATH}\\regioes.parquet")
df_regions = df_regions.withColumnRenamed("id", "id_regiao").withColumnRenamed("nome", "regiao")

# Escreve os dados no PostgreSQL usando SQL Alchemy e JDBC
df_regions.write \
    .jdbc(url=url_database, table="regiao", mode="overwrite", properties=properties)

In [6]:
# LER O PARQUE DE ESTADO E GRAVAR NO BANCO

df_state = spark_session.read.parquet(f"{PATH}\\estados.parquet")

df_state = df_state.withColumnRenamed("id","id_estado").withColumnRenamed("sigla","uf").withColumnRenamed("nome","estado")



# Escreve os dados no PostgreSQL 
df_state.select(
    'id_estado',
    'uf',
    'estado',
    col('regiao.id').alias('id_regiao')
    ).write \
    .jdbc(url=url_database, table="estado", mode="overwrite", properties=properties)

In [141]:
# LER O PARQUE DE MUNICIPIO E GRAVAR NO BANCO

df_substate = spark_session.read.parquet(f"{PATH}\\municipios.parquet")

df_substate = df_substate.withColumnRenamed("ibge", "id_municipio").withColumnRenamed("nome", "municipio")

# Escreve os dados no PostgreSQL 
df_substate.write \
    .jdbc(url=url_database, table="municipios", mode="overwrite", properties=properties)

In [14]:
# LE UM CSV COM AS CAPITAIS
df_capital = spark_session.read.csv(f"{PATH}\\capital.csv", header=True, inferSchema=True, sep=";", encoding="ISO-8859-1")

df_capital = df_capital.select(
    col("Estados").alias("estado"),
    col("Capital").alias("capital"),
    col("Sigla").alias("uf")
)

# Escreve os dados no PostgreSQL 
df_capital.write \
    .jdbc(url=url_database, table="capitais", mode="overwrite", properties=properties)


# PESQUISAS

In [142]:
# LER A PESQUISA IDH

subfolder = 'idh'
pathstring = f'{PATH}\\{subfolder}\\'


# Lista todos os arquivos Parquet na pasta
arquivos_parquet = [os.path.join(pathstring, arquivo) for arquivo in os.listdir(pathstring) if arquivo.endswith(".parquet")]

# Lê todos os arquivos Parquet e cria um DataFrame único
df_idh = spark_session.read.parquet(*arquivos_parquet)

print(df_idh.count())


# Escreve os dados no PostgreSQL 
df_idh.write \
    .jdbc(url=url_database, table="pesquisas_idh", mode="overwrite", properties=properties)


351


In [143]:
# LER A PESQUISA PIB

subfolder = 'pib'
pathstring = f'{PATH}\\{subfolder}\\'


# Lista todos os arquivos Parquet na pasta
arquivos_parquet = [os.path.join(pathstring, arquivo) for arquivo in os.listdir(pathstring) if arquivo.endswith(".parquet")]

# Lê todos os arquivos Parquet e cria um DataFrame único
df_pib = spark_session.read.parquet(*arquivos_parquet)

print(df_pib.count())


# Escreve os dados no PostgreSQL 
df_pib.write \
    .jdbc(url=url_database, table="pesquisas_pib", mode="overwrite", properties=properties)

22280


In [144]:
# LER A PESQUISA POPUPACAO

subfolder = 'populacao'
pathstring = f'{PATH}\\{subfolder}\\'


# Lista todos os arquivos Parquet na pasta
arquivos_parquet = [os.path.join(pathstring, arquivo) for arquivo in os.listdir(pathstring) if arquivo.endswith(".parquet")]

# Lê todos os arquivos Parquet e cria um DataFrame único
df_populacao = spark_session.read.parquet(*arquivos_parquet)

print(df_populacao.count())
# Escreve os dados no PostgreSQL 
df_populacao.write \
    .jdbc(url=url_database, table="pesquisas_populacao", mode="overwrite", properties=properties)

22280
